---
title: Classification
subtitle: Finding forests with satelite imagery
jupyter: 
  kernelspec:
    name: "01_classification"
    language: "python"
    display_name: "01_classification"
format: 
    html:
        code-fold: show
eval: true
---

## Data Acquisition
In this chapter, we will employ machine learning techniques to classify a scene using satellite imagery. Specifically, we will utilize ``scikit-learn`` to implement two distinct classifiers and subsequently compare their results. To begin, we need to import the following modules.

In [ ]:
from datetime import datetime, timedelta

import xarray as xr
import pystac_client
import stackstac
import odc.stac
import rioxarray
import geopandas as gpd
from odc.geo.geobox import GeoBox
from dask.diagnostics import ProgressBar
from rasterio.enums import Resampling
from shapely.geometry import Polygon, mapping

import cmcrameri as cmc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

# Scikit Learn
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import matplotlib.colors as colors

Before we start, we need to load the data. We will use ``odc-stac`` to obtain data from Earth Search by Element 84. Here we define the area of interest and the time frame, aswell as the EPSG code and the resolution.

### Searching Catalog
The module ``odc-stac`` provides access to free, open source satelite data. To retrieve the data, we must define  several parameters that specify the location and time period for the satellite data. Additionally, we must specify the data collection we wish to access, as multiple collections are available. In this example, we will use multispectral imagery from the Sentinel-2 satellite.

In [ ]:
dx = 0.0006  # 60m resolution
epsg = 4326

# Set Spatial extent
latmin, latmax = 47.86, 48.407
lonmin, lonmax = 16.32, 16.9
bounds = (lonmin, latmin, lonmax, latmax)
minx, miny, maxx, maxy = bounds
geom = {
    'type': 'Polygon',
    'coordinates': [[
       [minx, miny],
       [minx, maxy],
       [maxx, maxy],
       [maxx, miny],
       [minx, miny]
    ]]
}

# Set Temporal extent
year = 2024
month = 5
day = 1
delta = 10

start_date = datetime(year, month, day)
end_date = start_date + timedelta(days=delta)
date_query = start_date.strftime("%Y-%m-%d") + "/" + end_date.strftime("%Y-%m-%d")

# Search for Sentinel-2 data
items = pystac_client.Client.open(
    "https://earth-search.aws.element84.com/v1"
).search(
    intersects=geom,
    collections=["sentinel-2-l2a"],
    datetime=date_query,
    limit=100,
).item_collection()
print(len(items), 'scenes found')

We will now focus on the area south-east of Vienna, where the Nationalpark _Donauauen_ is situated. The time frame we are interested in is the beginning of May 2024.
After passing these parameters to the `stac-catalog` we have found **10 scenes** that we can use for our analysis. 

### Loading the Data
Now we will load the data directly into an ``xarray`` dataset, which we can use to perform computations on the data. ``xarray`` is a powerful library for working with multi-dimensional arrays, making it well-suited for handling satellite data.

Here's how we can load the data using odc-stac and xarray:

In [ ]:
# define a geobox for my region
geobox = GeoBox.from_bbox(bounds, crs=f"epsg:{epsg}", resolution=dx)

# lazily combine items
ds_odc = odc.stac.load(
    items,
    bands=["scl", "red", "green", "blue", "nir"],
    chunks={'time': 5, 'x': 600, 'y': 600},
    geobox=geobox,
    resampling="bilinear",
)

# actually load it
with ProgressBar():
    ds_odc.load()

## Data Visualization
### RGB Image
With the image data now in our possession, we can proceed with computations and visualizations.

First, we define a mask to exclude cloud cover and areas with missing data. Subsequently, we create a composite median image, where each pixel value represents the median value across all the scenes we have identified. This approach helps to eliminate clouds and outliers present in some of the images, thereby providing a clearer and more representative visualization of the scene.

In [ ]:
# define a mask for valid pixels (non-cloud)
def is_valid_pixel(data):
    # include only vegetated, not_vegitated, water, and snow
    return ((data > 3) & (data < 7)) | (data==11)

ds_odc['valid'] = is_valid_pixel(ds_odc.scl)
#ds_odc.valid.sum("time").plot()

def avg(ds):
    return (ds / ds.max() * 2)

# compute the masked median
rgb_median = (
    ds_odc[['red', 'green', 'blue']]
    .where(ds_odc.valid)
    .to_dataarray(dim="band")
    .transpose(..., "band")
    .median(dim="time")
    .astype(int)
)
rgb_comp = avg(rgb_median)
plot = rgb_comp.plot.imshow(rgb="band", figsize=(8, 8))
plot.axes.set_title(f"RGB - Median Composite\n{start_date.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')}")
plt.show()

### NDVI Image
To get an first impression of the data, we can calculate the NDVI (Normalized Difference Vegetation Index) and plot it. The NDVI is calculated by useing the following formula. [@rouse1974monitoring]

$$
NDVI = \frac{NIR - Red}{NIR + Red}
$$

This gives us a good overview of the vegetation in the area. The values can range from -1 to 1 where the following meanings are associated with these values:

- -1 to 0 indicate dead plants or inanimate objects
- 0 to 0.33 are unhealthy plants
- 0.33 to 0.66 are moderatly healthy plants
- 0.66 to 1 are very healthy plants

In [ ]:
# Normalized Difference Vegetation Index (NDVI)
def normalized_difference(a, b):
    return (a - b*1.) / (a + b)

ndvi = normalized_difference(ds_odc.nir, ds_odc.red)
ndvi.median(dim="time").plot.imshow(cmap='cmc.cork').axes.set_title('NDVI')
plt.show()

## Classification 

### Regions of Interest
Since this is a supervised classification, we need to have some training data. Therefore we need to define areas or regions, which we are certain represent the feature which we are classifiying. In this case we are looking for forested areas and areas that are definitly not forested. We will use these to train our classifiers. 

In [ ]:
# Define Polygons
forest_areas = {
    0: [Polygon([(16.482772, 47.901753), (16.465133, 47.870124), (16.510142, 47.874382), (16.482772, 47.901753)])],
    1: [Polygon([(16.594079, 47.938855), (16.581914, 47.894454), (16.620233, 47.910268), (16.594079, 47.938855)])],
    2: [Polygon([(16.67984, 47.978998), (16.637263, 47.971091), (16.660376, 47.929123), (16.67984, 47.978998)])],
    3: [Polygon([(16.756477, 48.000286), (16.723024, 47.983256), (16.739446, 47.972916), (16.756477, 48.000286)])],
    4: [Polygon([(16.80696, 48.135923), (16.780806, 48.125583), (16.798445, 48.115243), (16.80696, 48.135923)])],
    5: [Polygon([(16.684097, 48.144438), (16.664634, 48.124366), (16.690788, 48.118892), (16.684097, 48.144438)])],
    6: [Polygon([(16.550894, 48.169984), (16.530822, 48.165118), (16.558801, 48.137139), (16.550894, 48.169984)])],
    7: [Polygon([(16.588604, 48.402329), (16.556976, 48.401112), (16.580697, 48.382865), (16.588604, 48.402329)])],
}

nonforest_areas = {
    0: [Polygon([(16.674974, 48.269126), (16.623882, 48.236281), (16.682272, 48.213168), (16.674974, 48.269126)])],
    1: [Polygon([(16.375723, 48.228374), (16.357476, 48.188839), (16.399444, 48.185798), (16.375723, 48.228374)])],
    2: [Polygon([(16.457834, 48.26426), (16.418907, 48.267301), (16.440804, 48.23324), (16.457834, 48.26426)])],
    3: [Polygon([(16.519266, 48.101861), (16.470607, 48.100645), (16.500411, 48.07145), (16.519266, 48.101861)])],
    4: [Polygon([(16.453577, 48.051986), (16.412217, 48.067192), (16.425598, 48.012451), (16.453577, 48.051986)])],
}

# Geoppandas Dataframe from Polygons
forest_df = gpd.GeoDataFrame({'geometry': [poly[0] for poly in forest_areas.values()]}, crs="EPSG:4326")
nonforest_df = gpd.GeoDataFrame({'geometry': [poly[0] for poly in nonforest_areas.values()]}, crs="EPSG:4326")


# Plotting Regions of Interest
fig, ax = plt.subplots()
rgb_comp.plot.imshow(ax=ax)
forest_df.plot(ax=ax, ec='C0', fc='none')
nonforest_df.plot(ax=ax, ec='C1', fc='none')
ax.set_title('Regions of Interest')
ax.set_aspect('equal')
plt.show()

### Data Preparation
Additionally to the Regions of Interest we will extract the bands that we want to use for the classification from the loaded Dataset. With that we will create a Training and Testing Dataset, which we will train the classifier on.

In [ ]:
# Classifiying dataset (only necessary bands)
bands = ['red', 'green', 'blue', 'nir']
ds_class = (
    ds_odc[bands]
    .where(ds_odc.valid)
    .median(dim="time")
)
ds_class = avg(ds_class)
ds_class = ds_class.fillna(0)

def clip_array(ds:xr.Dataset, polygons):
    clipped = ds.rio.clip(polygons, invert=False, all_touched=False, drop=True)
    clipped_nan = clipped.where(clipped == ds)
    return clipped_nan

# Dictionaries with Dataarrays, each clipped by a Polygon
data_dict_feat = {idx: clip_array(ds_class, polygon) for idx, polygon in forest_areas.items()}
data_dict_nonfeat = {idx: clip_array(ds_class, polygon)  for idx, polygon in nonforest_areas.items()}

# Reshape the polygon dataarrays to get a tuple (one value per band) of pixel values
feat_data = [xarray.to_array().values.reshape(len(bands),-1).T for xarray in data_dict_feat.values()] # replaced median_data_dict_feat with data_dict_feat
nonfeat_data = [xarray.to_array().values.reshape(len(bands),-1).T for xarray in data_dict_nonfeat.values()] # replaced median_data_dict_feat with data_dict_feat

# The rows of the different polygons are concatenated to a single array for further processing
feat_values = np.concatenate(feat_data)
nonfeat_values = np.concatenate(nonfeat_data)

# Drop Nan Values
X_feat_data = feat_values[~np.isnan(feat_values).any(axis=1)]
X_nonfeat_data = nonfeat_values[~np.isnan(nonfeat_values).any(axis=1)]

# Creating Output Vector (1 for pixel is features; 0 for pixel is not feature)
y_feat_data = np.ones(X_feat_data.shape[0])
y_nonfeat_data = np.zeros(X_nonfeat_data.shape[0])

# Concatnate all Classes for training 
X = np.concatenate([X_feat_data, X_nonfeat_data])
y = np.concatenate([y_feat_data, y_nonfeat_data])

# Split into Training and Testing Data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

Now that we have the training and testing data, we create an Image array of the actual scene which we want to classify.

In [ ]:
image_data = ds_class[bands].to_array(dim='band').transpose('latitude', 'longitude', 'band')

# Reshape the image data
num_of_pixels = ds_class.sizes['longitude'] * ds_class.sizes['latitude']
num_of_bands = len(bands)
X_image_data = image_data.values.reshape(num_of_pixels, num_of_bands)

### Classifiying with Naive Bayes
Now that we have prepared all the needed data, we can start to classify the image.

We will start with a _Naive Bayes_ classificator. We train the classificator on our Training data and apply it on the actual image.


In [ ]:
# Naive Bayes initialization and training
nb = GaussianNB()
nb_test = nb.fit(X_train, y_train)
nb_predict = nb.predict(X_test)

# Prediction on image
nb_predict_img = nb.predict(X_image_data)
nb_predict_img = nb_predict_img.reshape(ds_class.sizes['latitude'], ds_class.sizes['longitude'])

# Adding the Naive Bayes Prediction to the dataset
ds_class['NB-forest'] = xr.DataArray(nb_predict_img, dims=['latitude', 'longitude'], coords={'longitude': ds_class['longitude'], 'latitude': ds_class['latitude']})

To see how well the classification has worked we plot the image that has been predicted by the classifier. Furthermore we can have a look at the `Classification Report` and the `Confusion Matrix`.  

In [ ]:
# Plot Naive Bayes
alpha = 1	
cmap_green = colors.ListedColormap([(1, 1, 1, alpha), 'green'])

plot = ds_class['NB-forest'].plot.imshow(cmap=cmap_green, cbar_kwargs={'ticks': [0.25,0.75]})
cbar = plot.colorbar
cbar.set_ticklabels(['non-forest', 'forest'])
plot.axes.set_title('Naive Bayes Classification')
plt.show()

# Print the Classification report
print("NAIVE BAYES: \n "+ classification_report(y_test, nb_predict))

# Print the confusion matrix
con_mat_nb = pd.DataFrame(confusion_matrix(y_test, nb_predict), 
                  index=['Actual Negative', 'Actual Positive'], 
                  columns=['Predicted Negative', 'Predicted Positive'])
display(con_mat_nb)

### Classifiying with Random Forest
To not only rely on one classificator lets have a look at another one. Here we use the _Random Forest_ Classificator. The Procedure useing it is the same as before.

In [ ]:
# Random Forest initialization and training
rf = RandomForestClassifier(n_estimators=100)
rf_test = rf.fit(X_train, y_train)
rf_predict = rf.predict(X_test)

# Prediction on image
rf_predict_img = rf.predict(X_image_data)
rf_predict_img = rf_predict_img.reshape(ds_class.sizes['latitude'], ds_class.sizes['longitude'])

# Adding the Random Forest Prediction to the dataset
ds_class['RF-forest'] = xr.DataArray(rf_predict_img, dims=['latitude', 'longitude'], coords={'longitude': ds_class['longitude'], 'latitude': ds_class['latitude']})

plot = ds_class['RF-forest'].plot.imshow(cmap=cmap_green, cbar_kwargs={'ticks': [0.25,0.75]})
cbar = plot.colorbar
cbar.set_ticklabels(['non-forest', 'forest'])
plot.axes.set_title('Random Forest Classification')
plt.show()

# Print the Classification report
print("RANDOM FOREST: \n "+ classification_report(y_test, rf_predict))

# Print the confusion matrix
con_mat_rf = pd.DataFrame(confusion_matrix(y_test, rf_predict), 
                  index=['Actual Negative', 'Actual Positive'], 
                  columns=['Predicted Negative', 'Predicted Positive'])
display(con_mat_rf)

We can already see from the `classification reports` and the `confusion matrices` that the _random forest_ classifier has performed better. This is for example indicated by the lower values in the secondary diagonal, which means that False Positvies and Negatives are only minimal. It seems that _Naive Bayes_ is more sensitive to False Positives.

### Comparison of the Classificators

To have a more in depth look at the performance of the classificators, we can compare them. Lets see what areas both classificators agree upon, and which areas then don't agree upon.

In [ ]:
#| code-fold: true
cmap_trio = colors.ListedColormap(['whitesmoke' ,'indianred', 'goldenrod', 'darkgreen'])


double_clf = (ds_class['NB-forest'] + 2*ds_class['RF-forest'])

fig, ax = plt.subplots()
cax = ax.imshow(double_clf, cmap=cmap_trio, interpolation='none')

# Add a colorbar with custom tick labels
cbar = fig.colorbar(cax, ticks=[1*0.375, 3*0.375, 5*0.375, 7*0.375])
cbar.ax.set_yticklabels(['None', 'Naive Bayes', 'Random Forest', 'Both'])
ax.set_title('Classification Comparisson')
ax.set_axis_off()
plt.show()

The areas that both agree upon are the bigger forests, like the _Nationalpark Donauauen_ and the _Leithagebirge_ also the urban areas of vienna have both rightfully not been classified.

In [ ]:
#| code-fold: true
# Plot only one class, either None (0), Naive Bayes (1), Random Forest (2), or Both (3)
fig, axs = plt.subplots(2,2, figsize=(8,8))
ax = axs.ravel()

for i in range(4):
    ax[i].imshow(double_clf==i, cmap='cmc.oleron_r', interpolation='none')
    category = ['by None', 'only by Naive Bayes', 'only by Random Forest', 'by Both'][i]
    title = 'Areas classified ' + category
    ax[i].set_title(title)
    ax[i].set_axis_off()

plt.tight_layout()

When plotting the areas, where classification has happend, individually we can see that the _random forest_ classifiyer falsly predicted the river _danube_ as a forest. On the other hand has the _naive bayes_ classifyer identified a lot of cropland as forest. Finally we can have a look at how big the percentage of forested areas in the scene are. We can see here that around 18% are forest and about 66% are not forest. The remaining areas are not so clear to define, as waterbodies and cropland are both in the remaining categories.

In [ ]:
#| code-fold: true
counts = {}
for num in range(0,4):
    num_2_class = {0: 'None', 1: 'Naive Bayes', 2: 'Random Forest', 3: 'Both'}
    counts[num_2_class[num]] = int((double_clf == num).sum().values)

class_counts_df = pd.DataFrame(list(counts.items()), columns=['Class', 'Count'])
class_counts_df['Percentage'] = (class_counts_df['Count'] / class_counts_df['Count'].sum())*100
ax = class_counts_df.plot.bar(x='Class', y='Percentage', rot=0, color='darkgreen', ylim=(0,100), title='Classified Areas per Classificator (%)')

# Annotate the bars with the percentage values
for p in ax.patches:
    ax.annotate(f'{p.get_height():.1f}%', (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', xytext=(0, 9), textcoords='offset points')